In [1]:
import os
import time
import pandas as pd
import numpy as np
import seaborn as sns
df = pd.read_csv('C:/Users/LENOVO/Desktop/downloads/ds_ready_data.csv', sep='\\t', engine='python')

In [4]:
# Очищаем названия колонок
df_clear_columns = df.rename(columns=lambda x: x.strip('"'))

In [5]:
def delete_marks(x):
    try:
        return x.str.strip('"')
    except:
        return x

In [6]:
# Очищаем значения в таблице
df_clear_columns = df_clear_columns.apply(lambda x: delete_marks(x))

In [8]:
# Меняем пробелы на NaN
df_without_marks_with_NaN = df_clear_columns.replace('', np.nan)

In [9]:
# Автоматически меняем тип объекта на дату
temp_option = pd.to_datetime(df_without_marks_with_NaN.ACTIVATION_DATE, infer_datetime_format=True)

In [10]:
df_without_marks_with_NaN.ACTIVATION_DATE = temp_option

In [12]:
search_copyes_df = df_without_marks_with_NaN.select_dtypes(include=['object']).copy()

In [14]:
# Оптимизируем таблицу, меняя тип столбцов
for col in search_copyes_df.columns:
    num_unique_values = len(search_copyes_df[col].unique())
    num_total_values = len(search_copyes_df[col])
    if num_unique_values / num_total_values < 0.5:
        df_without_marks_with_NaN.loc[:,col] = search_copyes_df[col].astype('category')        
    else:
        df_without_marks_with_NaN.loc[:,col] = search_copyes_df[col]

In [15]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174393 entries, 0 to 174392
Data columns (total 52 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   "ID_CONTACT                  174393 non-null  object 
 1   ""CAMPAIGN_SOURCE""          174393 non-null  object 
 2   ""PARTNER""                  174393 non-null  object 
 3   ""CAMPAIGN""                 174393 non-null  object 
 4   ""CAMPAIGN_MEDIUM""          174393 non-null  object 
 5   ""CAMPAIGN_TERM""            174393 non-null  object 
 6   ""CAMPAIGN_CONTENT""         174393 non-null  object 
 7   ""CAMPAIGN_NUMBER""          0 non-null       float64
 8   ""SEARCH_TERM""              0 non-null       float64
 9   ""AGE""                      174393 non-null  int64  
 10  ""SEX""                      0 non-null       float64
 11  ""IS_MARRIED""               0 non-null       float64
 12  ""IS_RETIRED""               0 non-null       float64
 13 

In [16]:
df_without_marks_with_NaN.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174393 entries, 0 to 174392
Data columns (total 52 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   ID_CONTACT               174393 non-null  object        
 1   CAMPAIGN_SOURCE          174373 non-null  category      
 2   PARTNER                  73655 non-null   category      
 3   CAMPAIGN                 171887 non-null  category      
 4   CAMPAIGN_MEDIUM          174393 non-null  category      
 5   CAMPAIGN_TERM            93862 non-null   category      
 6   CAMPAIGN_CONTENT         72681 non-null   category      
 7   CAMPAIGN_NUMBER          0 non-null       float64       
 8   SEARCH_TERM              0 non-null       float64       
 9   AGE                      174393 non-null  int64         
 10  SEX                      0 non-null       float64       
 11  IS_MARRIED               0 non-null       float64       
 12  IS_RETIRED      

In [17]:
# Сравниваем memory usage и видим результаты 299.4 MB => 117.2 MB

In [18]:
copy_df = df_without_marks_with_NaN.copy()

In [19]:
# Проверяем и удаляем пустые столбцы
for col in df_without_marks_with_NaN:
    if df_without_marks_with_NaN[col].isnull().sum() == 174393:
        copy_df = copy_df.drop(columns=col)

In [20]:
copy_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174393 entries, 0 to 174392
Data columns (total 37 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   ID_CONTACT               174393 non-null  object        
 1   CAMPAIGN_SOURCE          174373 non-null  category      
 2   PARTNER                  73655 non-null   category      
 3   CAMPAIGN                 171887 non-null  category      
 4   CAMPAIGN_MEDIUM          174393 non-null  category      
 5   CAMPAIGN_TERM            93862 non-null   category      
 6   CAMPAIGN_CONTENT         72681 non-null   category      
 7   AGE                      174393 non-null  int64         
 8   DT_APPLICATION_START     174393 non-null  category      
 9   LEAD_DATE                174393 non-null  category      
 10  LEAD_WEEK                174393 non-null  int64         
 11  LEAD_MONTH               174393 non-null  int64         
 12  CREDIT_LIMIT    

In [21]:
# Меняем регистр колонок
bank_df = copy_df.rename(columns=lambda x: x.lower())

In [22]:
campaigns = [i.lower().strip() for i in 'CAMPAIGN_SOURCE, CAMPAIGN, CAMPAIGN_MEDIUM, CAMPAIGN_TERM, CAMPAIGN_CONTENT'.split(',')]

In [25]:
# Отбираем только тех, кто создавал заявку на выдачу карты
only_issued = bank_df.loc[bank_df.issued.notna(), :]

In [26]:
only_issued = only_issued.reset_index(drop=True)

In [27]:
only_issued.query('issued == 1.0')

,id_contact,campaign_source,partner,campaign,campaign_medium,campaign_term,campaign_content,age,dt_application_start,lead_date,...,app_in_office_week,applied,issued,loan_amount,uniq,processed_with_delivery,online_decision,to_be_contacted,activated,activation_date
0,190081133,leadgid,NaN,24497,Affiliate,NaN,NaN,39,2018-04-25 21:50:00,2018-04-25,...,17.0,1.0,1.0,45000.0,1,0.0,"incomplete, score not requested, processed to CRM",1,1.0,2018-04-28 11:59:02
1,187705215,leads,NaN,52791,Network,52791},00bfc636ddb90cb5c6c5c28886d770ea,35,2018-04-19 14:25:00,2018-04-19,...,16.0,1.0,1.0,45000.0,1,0.0,"incomplete, score not requested, processed to CRM",1,0.0,NaT
4,182504990,leads,NaN,50817,Affiliate,50817,3d659da64a800ad817b2b5d6ab60609b,40,2018-04-05 15:00:00,2018-04-05,...,15.0,1.0,1.0,75000.0,1,0.0,"incomplete, score not requested, processed to CRM",1,0.0,NaT
6,189410820,smarter,NaN,halvaoffer,Callcenter,NaN,NaN,32,2018-04-24 11:00:23,2018-04-24,...,17.0,1.0,1.0,75000.0,1,1.0,"deferred, approved by dess",0,0.0,NaT
7,184536204,cityads,https://umllb.com/click-HQICWXOH-RMIQCEY7?bt=2...,55JZ,Network,29041-1-1523435360-4206157,NaN,47,2018-04-11 12:30:01,2018-04-11,...,16.0,1.0,1.0,30000.0,1,0.0,"incomplete, score not requested, processed to CRM",1,0.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23766,183021859,cityads,https://rfhhd.com/click-BQIOUJYG-RMIQCEY7?bt=2...,5m1w,Network,29041-1-1523017409-9224864,NaN,38,2018-04-06 16:25:00,2018-04-06,...,15.0,1.0,1.0,75000.0,1,0.0,"incomplete, score not requested, processed to CRM",1,0.0,NaT
23769,182173337,cityads,https://rfhhd.com/click-BQIOUJYG-RMIQCEY7?bt=2...,5m1w,Network,29041-1-1522856352-9202334,NaN,27,2018-04-04 19:40:00,2018-04-04,...,14.0,1.0,1.0,75000.0,1,0.0,"incomplete, score not requested, processed to CRM",1,1.0,2018-04-29 09:46:10
23770,191414037,saleads,NaN,881c0350-f5c6-11e7-99d4-7725e26f678f,Affiliate,9269a2c0-4b42-11e8-96a7-5756f7230fea,(not set),47,2018-04-29 03:17:04,2018-04-29,...,19.0,1.0,1.0,60000.0,1,NaN,NaN,1,1.0,2018-05-11 12:00:51
23774,189084337,mobilecredit,NaN,NaN,Mobile,NaN,NaN,118,2018-04-23 15:44:26,2018-04-23,...,17.0,1.0,1.0,30000.0,1,0.0,"incomplete, score not requested, processed to CRM",0,1.0,2018-04-23 19:24:59


In [28]:
# Создаём вырезку нужных столбцов
certain_df = only_issued[['campaign', 'campaign_medium', 'issued', 'id_contact', 'activated']]

In [30]:
# Считаем сколько вышло на один заказ Display
round(100000 / bank_df.query('campaign_medium == "Display"').shape[0])

267

In [31]:
# Далее меняем нужны типы на Объекты
certaion_df_without_na = certain_df.copy()

In [32]:
certaion_df_without_na.campaign = certain_df.campaign.astype('object').fillna(0)

In [33]:
certaion_df_without_na.campaign_medium = certain_df.campaign_medium.astype('object').fillna(0)

In [34]:
# Отбираем нужных провайдеров
provider = ['Agent', 'Affiliate', 'Network', 'PKW', 'Representative', 'Display']

In [35]:
certaion_df_without_na = certaion_df_without_na.query('campaign_medium in @provider')

In [36]:
df_required_provider = certaion_df_without_na.query('issued == 1.0') \
          .groupby(['campaign', 'campaign_medium'], as_index=False) \
          .agg({'id_contact': 'count'}) \
          .sort_values(by='id_contact', ascending=False) \
          .pivot(index='campaign', columns='campaign_medium', values='id_contact')
df_required_provider = df_required_provider.fillna(0).astype(int)

In [38]:
# Считаем сумму для каждой компании
for col_name in df_required_provider:
    column = df_required_provider[col_name]
    if col_name in ['Agent', 'Affiliate', 'Network']:
        df_required_provider[col_name] = df_required_provider[col_name] * 200
    elif col_name == 'PKW':
        df_required_provider[col_name] = df_required_provider[col_name] * 27
    elif col_name == 'Representative':
        df_required_provider[col_name] = df_required_provider[col_name] * 150
    elif col_name == 'Display':
        df_required_provider[col_name] = df_required_provider[col_name] * 267
        

In [40]:
df_required_provider['sum_rubs'] = df_required_provider.sum(axis=1)

In [41]:
df_required_provider.sort_values(by='sum_rubs', ascending=False)

campaign_medium,Affiliate,Agent,Display,Network,PKW,Representative,sum_rubs
campaign,,,,,,,
5m1w,0,0,0,196800,0,0,196800
27471,131000,0,0,0,0,0,131000
52218,0,0,0,115000,0,0,115000
24497,109800,0,0,0,0,0,109800
22797,97400,0,0,0,0,0,97400
...,...,...,...,...,...,...,...
2426210,0,0,0,0,0,150,150
1011105,0,0,0,0,0,150,150
facebook,0,0,0,0,54,0,54


In [42]:
# Добавляем стоимость всех PKW, а не только тех, которым выдали карты
find_all_PKW_df = certaion_df_without_na.query('issued == 0.0') \
          .groupby(['campaign', 'campaign_medium'], as_index=False) \
          .agg({'id_contact': 'count'}) \
          .sort_values(by='id_contact', ascending=False) \
          .pivot(index='campaign', columns='campaign_medium', values='id_contact')
find_all_PKW_df = find_all_PKW_df.fillna(0).astype(int)

In [44]:
find_all_PKW_df['PKW_SUM'] = find_all_PKW_df.PKW * 27

In [46]:
sum_PKW_df = df_required_provider.reset_index().merge(find_all_PKW_df.reset_index(), on='campaign', how='left')

In [48]:
sum_PKW_df.sum_rubs = sum_PKW_df.sum_rubs + sum_PKW_df.PKW_SUM

In [49]:
sum_rubs_compaign = sum_PKW_df[['campaign', 'sum_rubs']].sort_values(by='sum_rubs', ascending=False).reset_index(drop=True)

In [52]:
# Считаем сумму активированных карт по компаниям
sum_activated_cards = certain_df.query('activated == 1.0') \
          .groupby('campaign', as_index=False) \
          .agg({'id_contact': 'count'}) \
          .sort_values(by='id_contact', ascending=False) \
          .rename(columns={'id_contact': 'activated_count'})

In [54]:
final_df_for_step = sum_activated_cards.merge(sum_rubs_compaign.fillna(0), on='campaign')

In [55]:
# Высчитываем стоимость одной карты для каждой компании
final_df_for_step['one_card_cost'] = final_df_for_step.sum_rubs / final_df_for_step.activated_count

In [56]:
final_df_for_step.sort_values(by='one_card_cost')

,campaign,activated_count,sum_rubs,one_card_cost
110,200142,1,0.0,0.0
109,A_88andreypr,1,0.0,0.0
108,2408168,1,0.0,0.0
106,52795,1,0.0,0.0
103,907540,1,0.0,0.0
...,...,...,...,...
253,2424735,0,0.0,NaN
254,2420649,0,0.0,NaN
255,2408385,0,0.0,NaN
257,2408232,0,0.0,NaN


In [58]:
# Выводим только выданные карты
count_issued_df = certain_df.query('issued == 1.0') \
          .groupby('campaign', as_index=False) \
          .agg({'id_contact': 'count'}) \
          .query('id_contact > 0') \
          .rename(columns={'id_contact': 'count_issued'})

In [60]:
#Считаем сколько ВСЕГО было сделано заявок для каждой компании
count_campaign_leads = bank_df.groupby('campaign', as_index=False) \
       .campaign_source \
       .size()

In [61]:
# Джоиним в одну таблицу общее число заявок и выданных карт по компаниям
count_campaign_leads_final = count_issued_df.merge(count_campaign_leads, on='campaign')

In [62]:
# Высчитываем конверсию из лида в выдачу
count_campaign_leads_final['conversion_rate'] = count_campaign_leads_final['count_issued'] / count_campaign_leads_final['size'] * 100

In [63]:
# Объединяем по компаниям данные по конверсии и по сумме за одну карту
final_df_for_step_remade = final_df_for_step.merge(count_campaign_leads_final, on='campaign')

In [64]:
# Возьмём те компании, у которых примерно заявок больше среднего, больше 500
final_df_for_step_remade['size'].describe()

count      259.000000
mean       579.154440
std       1903.494153
min          1.000000
25%          3.500000
50%         25.000000
75%        166.500000
max      16807.000000
Name: size, dtype: float64

In [66]:
# Берём примерное количество заявок на компанию в размере 500
final_df_remade = final_df_for_step_remade.query('size > 500').sort_values(by='one_card_cost')

In [67]:
# Устанавливаем компании как индекс, для удобности дальнейшей работы
final_df_remade.set_index(final_df_remade['campaign'], inplace=True)

In [68]:
# Убираем колонку которая теперь в индексах
final_df_remade = final_df_remade.drop(columns=['campaign'])

In [69]:
# Этап объединения компании
final_df_remade.loc['GoogleAdWords', :] = final_df_remade.loc['GoogleAdWords':'GoogleAdwords'].sum()

In [70]:
final_df_remade = final_df_remade.drop(labels='GoogleAdwords')

In [71]:
# Устанавливаем ещё раз стоимость за одну карту и конверсию, для получившийся компании
final_df_remade['one_card_cost'] = final_df_remade.sum_rubs / final_df_remade.activated_count

In [72]:
final_df_remade['conversion_rate'] = round(final_df_remade.count_issued / final_df_remade['size'] * 100, 2)

In [74]:
# Вычисляем минимальный KPI по конверсии из лида в выдачу (issued)
min_KPI = only_issued.query('issued == 1.0').shape[0] / bank_df.shape[0] * 100

In [75]:
min_KPI = round(min_KPI, 2)

In [77]:
# Создаём таблицы с лучшими и худшими компаниями по цене одной карты
high_quality = final_df_remade.query('conversion_rate > @min_KPI')
low_quality = final_df_remade.query('conversion_rate < @min_KPI')

In [78]:
high_quality = high_quality[['one_card_cost','conversion_rate']]
high_quality['qualitative'] = True

In [79]:
low_quality = low_quality[['one_card_cost','conversion_rate']]
low_quality['qualitative'] = False

In [80]:
# Объединяем таблицы для отправки csv
final_df = pd.concat([high_quality, low_quality])

In [81]:
final_df.to_csv('remade_test_bank')